In [30]:
import os

In [31]:
%pwd

'c:\\Users\\kanha'

In [39]:
os.chdir("../")

In [47]:
%cd C:/Users/kanha/Chicken_Disease_Classification_Project


C:\Users\kanha\Chicken_Disease_Classification_Project


c:\Users\kanha\anaconda3\envs\Chickendisease\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [48]:
%pwd

'C:\\Users\\kanha\\Chicken_Disease_Classification_Project'

In [104]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_name])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])


        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        ) 
        
        return data_ingestion_config


In [107]:
import os
import urllib.request as request
import zipfile
from CNNClassifier import logger
from CNNClassifier.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config




    def download_file(self):

        file_path = Path(self.config.local_data_file)
        min_valid_size_kb = 50  # adjust depending on expected dataset size

        if not file_path.exists() or file_path.stat().st_size < min_valid_size_kb * 1024:
            if file_path.exists():
                logger.info(f"File exists but is too small ({file_path.stat().st_size // 1024} KB), re-downloading...")
                os.remove(file_path)

            filename, headers = request.urlretrieve(
            url=self.config.source_URL,
            filename=str(file_path)
        )
        logger.info(f"{filename} downloaded successfully with info:\n{headers}")
    else:
        logger.info(f"File already exists of size: {file_path.stat().st_size // 1024} KB")
  



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """ 

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)




In [109]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e    

[2025-08-14 10:13:14,269: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2025-08-14 10:13:14,274: INFO: common: yaml file: params.yaml loaded succesfully]
[2025-08-14 10:13:14,277: INFO: common: created directory at: artifacts]
[2025-08-14 10:13:14,278: INFO: common: created directory at: artifacts/data_ingestation]
[2025-08-14 10:13:14,280: INFO: 1678127896: File exists but is too small (0 KB), re-downloading...]


[2025-08-14 10:13:22,589: INFO: 1678127896: artifacts\data_ingestation\data.zip downloaded successfully with info:
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7EA6:157CDF:29B76:755FF:689D6964
Accept-Ranges: bytes
Date: Thu, 14 Aug 2025 04:43:18 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830041-CCU
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1755146598.767313,VS0,VE914
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 1b732f0010bd9dbff631f4cca6063daf10c259e5
Expires: Thu, 14 Aug 2025 04:48:18 GMT
Source-Age: 1

]
